In [4]:
%run "./Includes/Classroom-Setup"

### The Case for Packaging

There are a number of different reasons why teams need to package their machine learning projects:<br><br>

1. Projects have various library dependencies so shipping a machine learning solution involves the environment in which it was built.  MLflow allows for this environment to be a conda environment or docker container.  This means that teams can easily share and publish their code for others to use.
2. Machine learning projects become increasingly complex as time goes on.  This includes ETL and featurization steps, machine learning models used for pre-processing, and finally the model training itself.
3. Each component of a machine learning pipeline needs to allow for tracing its lineage.  If there's a failure at some point, tracing the full end-to-end lineage of a model allows for easier debugging.

**ML Projects is a specification for how to organize code in a project.**  The heart of this is an **MLproject file,** a YAML specification for the components of an ML project.  This allows for more complex workflows since a project can execute another project, allowing for encapsulation of each stage of a more complex machine learning architecture.  This means that teams can collaborate more easily using this architecture.

### Packaging a Simple Project

First we're going to create a simple MLflow project consisting of the following elements:<br><br>

1. MLProject file
2. Conda environment
3. Basic machine learning script

We're going to want to be able to pass parameters into this code so that we can try different hyperparameter options.

Create a new experiment for this exercise.  Navigate to the UI in another tab.

In [9]:
experimentPath = "/Users/" + username + "/experiment-SPL3"

In [10]:
import mlflow

mlflow.set_experiment(experimentPath)
mlflow_client = mlflow.tracking.MlflowClient()
experimentID = mlflow_client.get_experiment_by_name(name=experimentPath).experiment_id

print(f"The experiment can be found at the path `{experimentPath}` and has an experiment_id of `{experimentID}`")

INFO: '/Users/jose.manuel.bustos.munoz@everis.com/experiment-SPL3' does not exist. Creating a new experiment
The experiment can be found at the path `/Users/jose.manuel.bustos.munoz@everis.com/experiment-SPL3` and has an experiment_id of `33850450030010`

First, examine the code we're going to run.  This looks similar to what we ran in the last lesson with the addition of decorators from the `click` library.  This allows us to parameterize our code.

In [12]:
import click
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

@click.command()
@click.option("--data_path", default="/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv", type=str)
@click.option("--n_estimators", default=10, type=int)
@click.option("--max_depth", default=20, type=int)
@click.option("--max_features", default="auto", type=str)
def mlflow_rf(data_path, n_estimators, max_depth, max_features):

  with mlflow.start_run() as run:
    # Import the data
    df = pd.read_csv(data_path)
    X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)
    
    # Create model, train it, and create predictions
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Log model
    mlflow.sklearn.log_model(rf, "random-forest-model")
    
    # Log params
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("max_features", max_features)

    # Log metrics
    mlflow.log_metric("mse", mean_squared_error(y_test, predictions))
    mlflow.log_metric("mae", mean_absolute_error(y_test, predictions))  
    mlflow.log_metric("r2", r2_score(y_test, predictions))  

# if __name__ == "__main__":
#   mlflow_rf() # Note that this does not need arguments thanks to click

Test that it works using the `click` `CliRunner`, which will execute the code in the same way we expect to.

In [14]:
from click.testing import CliRunner

runner = CliRunner()
result = runner.invoke(mlflow_rf, ['--n_estimators', 10, '--max_depth', 20], catch_exceptions=True)

assert result.exit_code == 0, "Code failed" # Check to see that it worked

print("Success!")

Success!

Now create a directory to hold our project files.  This will be a unique directory for this lesson.

In [16]:
# Adust our working directory from what DBFS sees to what python actually sees
working_path = workingDir.replace("dbfs:", "/dbfs")

Create the `MLproject` file.  This is the heart of an MLflow project.  It includes pointers to the conda environment and a `main` entry point, which is backed by the file `train.py`.

Any `.py` or `.sh` file can be an entry point.

In [18]:
dbutils.fs.put(f"{workingDir}/MLproject", 
'''
name: Lesson-3-Model-Training

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      data_path: {type: str, default: "/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv"}
      n_estimators: {type: int, default: 10}
      max_depth: {type: int, default: 20}
      max_features: {type: str, default: "auto"}
    command: "python train.py --data_path {data_path} --n_estimators {n_estimators} --max_depth {max_depth} --max_features {max_features}"
'''.strip())

Wrote 471 bytes.
Out[18]: True

Create the conda environment.

You can also dynamically view and use a package version by calling `.__version__` on the package.

In [20]:
import cloudpickle, numpy, pandas, sklearn

file_contents = f"""
name: Lesson-03
channels:
  - defaults
dependencies:
  - cloudpickle={cloudpickle.__version__}
  - numpy={numpy.__version__}
  - pandas={pandas.__version__}
  - scikit-learn={sklearn.__version__}
  - pip:
    - mlflow=={mlflow.__version__}
""".strip()

dbutils.fs.put(f"{workingDir}/conda.yaml", file_contents, overwrite=True)

print(file_contents)

Wrote 162 bytes.
name: Lesson-03
channels:
 - defaults
dependencies:
 - cloudpickle=1.3.0
 - numpy=1.16.2
 - pandas=0.24.2
 - scikit-learn=0.20.3
 - pip:
 - mlflow==1.7.2

Now create the code itself.  This is the same as above except for with the `__main__` is included.  Note how there are no arguments passed into `mlflow_rf()` on the final line.  `click` is handling the arguments for us.

In [22]:
dbutils.fs.put(f"{workingDir}/train.py", 
'''
import click
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

@click.command()
@click.option("--data_path", default="/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv", type=str)
@click.option("--n_estimators", default=10, type=int)
@click.option("--max_depth", default=20, type=int)
@click.option("--max_features", default="auto", type=str)
def mlflow_rf(data_path, n_estimators, max_depth, max_features):

  with mlflow.start_run() as run:
    # Import the data
    df = pd.read_csv(data_path)
    X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)
    
    # Create model, train it, and create predictions
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Log model
    mlflow.sklearn.log_model(rf, "random-forest-model")
    
    # Log params
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("max_features", max_features)

    # Log metrics
    mlflow.log_metric("mse", mean_squared_error(y_test, predictions))
    mlflow.log_metric("mae", mean_absolute_error(y_test, predictions))  
    mlflow.log_metric("r2", r2_score(y_test, predictions))  

if __name__ == "__main__":
  mlflow_rf() # Note that this does not need arguments thanks to click
'''.strip(), True)

Wrote 1610 bytes.
Out[20]: True

To summarize, you now have three files: `MLproject`, `conda.yaml`, and `train.py`

In [24]:
display( dbutils.fs.ls(workingDir) )

path,name,size
dbfs:/user/jose.manuel.bustos.munoz@everis.com/mlflow/03_packaging_ml_projects_psp/MLproject,MLproject,471
dbfs:/user/jose.manuel.bustos.munoz@everis.com/mlflow/03_packaging_ml_projects_psp/conda.yaml,conda.yaml,162
dbfs:/user/jose.manuel.bustos.munoz@everis.com/mlflow/03_packaging_ml_projects_psp/train.py,train.py,1610


### Running Projects

Now you have the three files we need to run the project, we can trigger the run.  We'll do this in a few different ways:<br><br>

1. On the driver node of our Spark cluster
2. On a new Spark cluster submitted as a job
3. Using files backed by GitHub

Now run the experiment.  This command will execute against the driver node of a Spark cluster, though it could be running locally or on a different remote VM.

First set the experiment using the `experimentPath` defined earlier.  Prepend `/dbfs` to the file path, which allows the cluster's file system to access DBFS.  Then, pass your parameters.

This will take a few minutes to build the environment for the first time.  Subsequent runs are faster since `mlflow` can reuse the same environment after it has been built.

In [27]:
import mlflow

mlflow.projects.run(uri=working_path,
  parameters={
    "data_path": "/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv",
    "n_estimators": 10,
    "max_depth": 20,
    "max_features": "auto"
})

Out[22]: <mlflow.projects.submitted_run.LocalSubmittedRun at 0x7f8c491c1668>

Check the run in the UI.  Notice that you can see the run command.  **This is very helpful in debugging.**

Now that it's working, experiment with other parameters.

In [29]:
mlflow.projects.run(working_path,
  parameters={
    "data_path": "/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv",
    "n_estimators": 1000,
    "max_depth": 10,
    "max_features": "log2"
})

Out[23]: <mlflow.projects.submitted_run.LocalSubmittedRun at 0x7f8c491c1e48>

How did the new model do?

Now try executing this code against a new Databricks cluster.  This needs cluster specifications in order for Databricks to know what kind of cluster to use.  Uncomment the following to run it.

In [32]:
# clusterspecs = {
#     "num_workers": 2,
#     "spark_version": "5.2.x-scala2.11",
#     "spark_conf": {},
#     "aws_attributes": {
#         "first_on_demand": 1,
#         "availability": "SPOT_WITH_FALLBACK",
#         "zone_id": "us-west-1c",
#         "spot_bid_price_percent": 100,
#         "ebs_volume_count": 0
#     },
#     "node_type_id": "i3.xlarge",
#     "driver_node_type_id": "i3.xlarge"
#   }

# mlflow.projects.run(
#   uri=working_path,
#   parameters={
#     "data_path": "/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv",
#     "n_estimators": 1500,
#     "max_depth": 5,
#     "max_features": "sqrt"
# },
#   backend="databricks",
#   backend_config=clusterspecs
# )

Finally, run this example, which is <a href="https://github.com/mlflow/mlflow-example" target="_blank">a project backed by GitHub.</a>

In [34]:
mlflow.run(
  uri="https://github.com/mlflow/mlflow-example",
  parameters={'alpha':0.4}
)

Out[24]: <mlflow.projects.submitted_run.LocalSubmittedRun at 0x7f8c491a0f60>

## Review

**Question:** Why is packaging important?  
**Answer:** Packaging not only manages your code but the environment in which it was run.  This environment can be a Conda or Docker environment.  This ensures that you have reproducible code and models that can be used in a number of downstream environments.

**Question:** What are the core components of MLflow projects?  
**Answer:** An MLmodel specifies the project components using YAML.  The environment file contains specifics about the environment.  The code itself contains the steps to create a model or process data.

**Question:** What code can I run and where can I run it?  
**Answer:** Arbitrary code can be run in any number of different languages.  It can be run locally or remotely, whether on a remote VM, Spark cluster, or submitted as a Databricks job.

In [37]:
%run "./Includes/Classroom-Cleanup"